In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
### Importing libraries
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
import os

import keras
from keras.layers import *
from keras.models import *
import pickle

In [ ]:
### dataset path
data_path="../input/actigraphy/data"
condition_path="../input/actigraphy/data/condition"
control_path="../input/actigraphy/data/control"

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
### Preparing Dataset
X_data=[]
Y_data=[]

for filenames in os.listdir(condition_path):
    path = os.path.join(condition_path, filenames)
    data_frame=pd.read_csv(path)
    data_frame=data_frame.filter(['activity'])
    data_frame=np.array(data_frame)
    n=data_frame.shape
    len=n[0]
    quo=len/180
    quo=int(quo)
    new_len= quo*180
    lst=data_frame.tolist()
    lst=lst[:new_len]
    new_arr=np.array_split(lst, quo)
    for i in range(0,quo):
      X_data.append(new_arr[i])
      Y_data.append(1)  

In [ ]:
for filenames in os.listdir(control_path):
    path = os.path.join(control_path, filenames)
    data_frame=pd.read_csv(path)
    data_frame=data_frame.filter(['activity'])
    data_frame=np.array(data_frame)
    n=data_frame.shape
    len=n[0]
    quo=len/180
    quo=int(quo)
    new_len= quo*180
    lst=data_frame.tolist()
    lst=lst[:new_len]
    new_arr=np.array_split(lst, quo)
    for i in range(0,quo):
      X_data.append(new_arr[i])
      Y_data.append(0)


In [ ]:
X_data=np.array(X_data)
print(X_data.shape)
Y_data=np.array(Y_data)
print(Y_data.shape)


In [ ]:
# np.savez_compressed('X_data.npz',np.array(X_data))
# np.savez_compressed('Y_data',np.array(Y_data))

# X_data = np.array(np.load('/content/X_data.npz',allow_pickle=True)['arr_0'],dtype=np.float16)
# Y_data = np.array(np.load('/content/Y_data.npz',allow_pickle=True)['arr_0'],dtype=np.float16)

In [ ]:
X_data,Y_data = shuffle(X_data,Y_data)

In [ ]:
X_train=X_data[:6903]
y_train=Y_data[:6903]
X_test=X_data[6903:]
y_test=Y_data[6903:]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
with tpu_strategy.scope():
    num_layers = 2
    d_model = 128
    num_heads = 8
    dff = 256
    max_seq_len = 180 #X_train.shape[1]
    pe_input = 250
    rate = 0.3
    num_features = 1

    ###### Defining Layers
    Input_layer = tf.keras.layers.Input(shape=(max_seq_len,num_features))

    ##### Convolutional Filters

    ### Layer-1
    conv1 = tf.keras.layers.Conv1D(32,15,padding='same',activation='relu')
    conv11 = tf.keras.layers.Conv1D(32,15,padding='same',activation='relu')
    conv12 = tf.keras.layers.Conv1D(32,15,padding='same',activation='relu')
    conv13 = tf.keras.layers.Conv1D(32,15,padding='same',activation='relu')

    ### Layer-2
    conv2 = tf.keras.layers.Conv1D(64,15,padding='same',activation='relu')
    conv21 = tf.keras.layers.Conv1D(64,15,padding='same',activation='relu')
    conv22 = tf.keras.layers.Conv1D(64,15,padding='same',activation='relu')
    conv23 = tf.keras.layers.Conv1D(64,15,padding='same',activation='relu')

    ### Layer-3
    conv3 = tf.keras.layers.Conv1D(128,15,padding='same',activation='relu')
    conv31 = tf.keras.layers.Conv1D(128,15,padding='same',activation='relu')
    conv32 = tf.keras.layers.Conv1D(128,15,padding='same',activation='relu')
    conv33 = tf.keras.layers.Conv1D(128,15,padding='same',activation='relu')

    #### Channel Attention Module
    # cam_module = CAM_Module(128,1)

    ##### Transfromer Layer
    #transformer = Transformer(num_layers,d_model,num_heads,dff,pe_input,rate)

    ##### Output Layer
    gap_layer = tf.keras.layers.GlobalAveragePooling1D()

    ###### Defining Architecture
    ##### Input Layer
    Inputs = Input_layer

    ##### Network
    #### Layer-1
    conv1_up = conv1(Inputs)
    conv_11 = conv11(conv1_up) 
    conv_12 = conv12(conv_11)
    conv_13 = conv13(conv_12)
    conv_13 = tf.keras.layers.Add()([conv_13,conv_11])

    #### Layer-2
    conv2_up = conv2(conv_13)
    conv_21 = conv21(conv2_up)
    conv_22 = conv22(conv_21)
    conv_23 = conv23(conv_22)
    conv_23 = tf.keras.layers.Add()([conv_23,conv_21])

    #### Layer-3
    conv3_up = conv3(conv_23)
    conv_31 = conv31(conv3_up)
    conv_32 = conv32(conv_31)
    conv_33 = conv33(conv_32)
    conv_33 = tf.keras.layers.Add()([conv_33,conv_31])

    ##### Transformer
    #embeddings =  transformer(inp=conv_33,enc_padding_mask=None)

    ##### CAM Module
    # cam_op = cam_module(conv_33)
    # cam_op = tf.keras.layers.Add()([cam_op,embeddings])

    ##### Output Layers
    #### Initial Layers
    gap_op = gap_layer(conv_33)
    dense1 = tf.keras.layers.Dense(256,activation='relu')(gap_op)
    dropout1 = tf.keras.layers.Dropout(0.3)(dense1)
    dense2 = tf.keras.layers.Dense(256,activation='relu')(dropout1)
    dense3 = tf.keras.layers.Dense(1,activation='sigmoid')(dense2)

    ##### Compiling Architecture            
    model = tf.keras.models.Model(inputs=Inputs,outputs=dense3)
    # model.load_weights('./FECGCL_1.h5')
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),loss='binary_crossentropy',metrics=['accuracy'])
# model.load_weights('../input/weights/actigraphy_wts.hdf5')

In [ ]:
##### Compiling Architecture            
model = tf.keras.models.Model(inputs=Inputs,outputs=dense3)
# model.load_weights('./FECGCL_1.h5')
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),loss='binary_crossentropy',metrics=['accuracy'])

model.summary()      
tf.keras.utils.plot_model(model)

###### Model Training 
##### Model Checkpointing

filepath = './FECGCL_1.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,monitor='val_accuracy',save_best_only=True,mode='max',save_weights_only=True)

In [ ]:
##### Softmax Training 
history = model.fit(X_train,y_train,epochs=100,batch_size=128,validation_data=(X_train,y_train),validation_batch_size=128,callbacks=checkpoint)

In [ ]:
#### Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')  
plt.legend(['Train', 'Testing'], loc='best')
plt.show()

### Loss     
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])
plt.title('Model Loss')  
plt.ylabel('Loss')         
plt.xlabel('epoch')
plt.legend(['Train', 'Testing'], loc='best')   
plt.show()   

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_test[1].shape

In [ ]:
single_pred=model.predict(X_test[1])
X_axis=[]
for i in range(180):
    X_axis.append(i)
X_axis=np.array(X_axis)

In [ ]:
plt.plot(X_axis, single_pred)

In [ ]:
### Data of a single person
single_person=[]
path= "../input/actigraphy/data/condition/condition_10.csv"
df=pd.read_csv(path)
df=df.filter(['activity'])
df=np.array(df)
n=df.shape
len=n[0]
quo=len/180
quo=int(quo)
new_len= quo*180
lst=df.tolist()
lst=lst[:new_len]
new_arr=np.array_split(lst, quo)
for i in range(0,quo):
    single_person.append(new_arr[i])

In [ ]:
single_person=np.array(single_person)
single_person.shape

In [ ]:
model.predict(single_person)

In [ ]:
y_preds=model.predict(X_test)

In [ ]:
y_preds=np.round(y_preds)

In [ ]:
y_preds
y_preds = y_preds. astype(int)
y_preds

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_preds)

In [ ]:
plt.rcParams["figure.figsize"] = [6,6]
import itertools
def plot_confusion_matrix(cm,classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
###### Confusion Matrix Computation
cm = confusion_matrix(y_test,np.round(y_preds))
print('//////////////////////////////////')
print(cm)
print('//////////////////////////////////')

###### Plotting Confusion Matix
cm_plot_labels = ['0','1']
plot_confusion_matrix(cm=cm,classes=cm_plot_labels,normalize=False,title='Confusion Matrix')

In [ ]:
cm